In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Minimal SVM training for feature selection

# We'll use a simple StandardScaler + RBF SVM pipeline
def svm_pipeline(C, gamma):
    return Pipeline([
        ("scaler", StandardScaler()),
        ("svm", SVC(kernel="rbf", C=C, gamma=gamma, random_state=42))
    ])

#------------

#nel nostro caso possiamo fare una roba del genere (fuori dal ciclo for dove iteriamo su gamma e c)
pipeline = Pipeline([("scaler" , StandardScaler()) , ("svm" , SVC())])
# e poi all'interno di ogni iterazione
pipeline.set_params(svm__kernel="rbf",svm__C=C , svm__gamma=gamma) #per settare i parametri della svm, volendo si può iterare anche sul kernel
#con set params possiamo anche cambiare i parametri dello standard scaler
pipeline.fit(X_train, y_train) #addestra lo scaler e l'svm in base ai dati. Nel caso dello scaler non è che impara qualcosa veramente come nel caso 
#dell'svm o di modelli predittivi in generale. 
#Ma bensi memorizza calcola e memorizza mean e stdv, da usare poi scaler.transform(X) per trasformare questi dati. 
score = pipeline.score(X_test, y_test)

la funzione pipeline serve semplicemente per stabilire un ordine di cose da fare (nel caso sopra scale e creare la svm)
cosi che puoi successivamente riciclare la tua funzione pipeline ad esempio per usarla in diversi dataset.
viene usata per la cross validation. Non è necessario definire una funzione come fa il prof. Si può direttamente usare pipeline
IMPORTANTE: pipeline.score returna l'accuracy. Questo per noi non va bene visto che abbiamo classi sbilanciate.
quello che possiamo fare allora è il seguente:

In [ ]:
from sklearn.metrics import matthews_corrcoef

y_pred = pipeline.predict(X_test) #che cos'è sta roba??? a noi non serve in teoria abbiamo gia y_pred o no?? -CUS
mcc = matthews_corrcoef(y_test, y_pred)
# e usare l'mcc come score invece che l'accuracy

In [2]:
# for loop per la grid search
C_grid = [0.1, 1.0, 10.0, 100.0]
gamma_grid = ["scale", 0.01, 0.1, 1.0]

best_score_base = -np.inf
best_params_base = None

for C in C_grid:
    for gamma in gamma_grid:
        pipe = svm_pipeline(C, gamma)
        pipe.fit(X_train, y_train)                 # fit on TRAIN
        val_acc = pipe.score(X_val, y_val)         # evaluate on VALIDATION
        if val_acc > best_score_base:
            best_score_base = val_acc
            best_params_base = {"C": C, "gamma": gamma}

print("Baseline SVM (all features) — best validation accuracy: "
      f"{best_score_base:.3f} with params {best_params_base}")

in questo step c'è semplicemente la grid search per trovare il miglior C e gamma. Possiamo modificare il codice come ho scritto sopra

In [ ]:
# for loop per la grid search modificato
pipeline = Pipeline([("scaler" , StandardScaler()) , ("svm" , SVC())])

C_grid = [0.1, 1.0, 10.0, 100.0]
gamma_grid = ["scale", 0.01, 0.1, 1.0]

best_mcc = -1
best_params_base = None

for C in C_grid:
    for gamma in gamma_grid:
        pipeline.set_params(svm__kernel="rbf",svm__C=C , svm__gamma=gamma)
        pipe.fit(X_train, y_train)                 # fit on TRAIN
        y_pred = pipeline.predict(X_test)
        mcc = matthews_corrcoef(y_test, y_pred)      
        if mcc > best_mcc:
            best_mcc = mcc
            best_params_base = {"C": C, "gamma": gamma}

print("Baseline SVM (all features) — best validation accuracy: "
      f"{best_score_base:.3f} with params {best_params_base}")

fin adesso semplcemente abbiamo settato dei parametri ottimale per un svm base che possiamo usare in seguito per testare le features selezionate dalla random forest. Quindi adesso dobbiamo fare la random forest

In [ ]:
rf = RandomForestClassifier(
    n_estimators=400,
    n_jobs=-1
) 
#costruttore della random forest. n_jobs si riferisce al numero di operazioni .fit da parallelizzare (visto che gli alberi sono indipendenti)
# -1 vuol dire utilizzare tutti i processori in parallelo. 

rf.fit(X_train, y_train)  # fit only on TRAIN

gini_imp = pd.Series(rf.feature_importances_, index=feature_names).sort_values(ascending=False)
# la funzione feature_importances_ calcola l'importanza delle features basandosi sull impurità

# da qui in poi è semplicemente plotting e cazzate varie. 
gini_df = gini_imp.reset_index()
gini_df.columns = ["feature", "importance"]
print("Top 10 features by Gini importance:")
print(gini_df.head(10))

# Plot top 20
plt.figure()
plt.barh(gini_df["feature"].head(20)[::-1], gini_df["importance"].head(20)[::-1])
plt.xlabel("Gini importance")
plt.ylabel("Feature")
plt.title("RandomForest Gini Importances (Top 20)")
plt.show()

IMPORTANTE " impurity-based feature importance for trees is strongly biased and favor high cardinality features (typically numerical features) over low cardinality features such as binary features or categorical variables with a small number of possible categories.

Permutation-based feature importances do not exhibit such a bias."
questo ce scritto sul sito di scikit-learn. Permutation-based feature importances sono un altro modo per calcolare l'importanza delle features usando RandomForest ma senza basarsi sull impurità. Se nel nostro dataset ci sono sia feature prettamente numeriche e feature categoriche, allora ci conviene usare il secondo metodo, altrimenti anche quello del prof va bene.

In [ ]:
def accuracy_on_subset(C, gamma, subset_features):
    # subset by feature names
    idx = [np.where(feature_names == f)[0][0] for f in subset_features]
    # questa sintassi terribile seleziona solo le feature in subset_features
    # np.where resistuisce le posizioni dove feature_names è uguale a f (che sono le fatures in subset features
    # feature names è un array (ma anche una lista va bene) che contiene i nomi delle features, cosi da poterne tenere traccia.
    Xtr = X_train[:, idx]
    Xva = X_val[:, idx]
    # con sta roba prendiamo solo le colonne che ci interessano
    pipe = svm_pipeline(C, gamma)
    pipe.fit(Xtr, y_train)     # train on TRAIN only
    y_pred = pipeline.predict(x_test) # predict on test data
            mcc = matthews_corrcoef(y_test, y_pred) # compute MCC 
    return pipe.score(Xva, y_val)  # accuracy on VALIDATION

#possiamo modificare le ultime righe di questo codice se utilizzano la pipeline come ho detto io. Possiamo anche 
#qui usare un altra metric invece dell accuracy 



# We'll sweep k and, for each k, re-evaluate the best baseline SVM params on the reduced feature set
ks = list(range(2, min(26, X_train.shape[1]+1)))  # keep it small for speed/clarity
curve = []

for k in ks:
    subset = gini_df["feature"].head(k).tolist()
    acc_k = accuracy_on_subset(best_params_base["C"], best_params_base["gamma"], subset)
    curve.append(acc_k)

#gini_df contiene una classifica di feature ordinate per importanza Gini (probabilmente da una Random Forest).
#Per ogni k (da 2 a max 25 o numero di feature disponibili):
#Prende le prime k feature più importanti (head(k)).
#Calcola l’accuracy sul validation set usando solo quelle feature.
#Salva il risultato in curve.
# questo l ha fatto chat pk mi sono rotto il cazzo, comunque in questo script prende quindi , 
# prima la feature piu importante, poi la prima e la seconda, poi prima seconda e terza, finche non le usa tutte.
#ogni volta calcola l accuracy dopo aver provato la SVM base di prima. 

best_k_idx = int(np.argmax(curve))
#argmax ti returna l indice del valore piu alto in quella lista.
best_k = ks[best_k_idx]
print(f"Best k on validation (using baseline best params): k={best_k}, val_acc={curve[best_k_idx]:.3f}")

#trova k che massimizza la curva, poi solo plotting 
plt.figure()
plt.plot(ks, curve, marker="o")
plt.xlabel("k (top features by RF Gini)")
plt.ylabel("Validation accuracy (SVM)")
plt.title("Accuracy vs. Number of Selected Features (Validation set)")
plt.grid(True)
plt.show()

PARTE FINALE: ALLENARE L'SVM FINALE

Dobbiamo fare due cose: di nuovo una grid search per scegliere i parametri migliori.
E dobbiamo scegliere quali e quante feauture usare.
dobbiamo guardare tutti i parametri che ci ha dato la cross validation
possiamo mettere in una lista tutti i valori migliori di k e testarli uno ad uno sul testing set e poi scegliere il migliore.

In [ ]:
# Use the best k from the validation curve
# --> nel nostro caso abbiamo piu k e dobbiamo iterare
best_subset = gini_df["feature"].head(best_k).tolist()
idx = [np.where(feature_names == f)[0][0] for f in best_subset]

Xtr_sel = X_train[:, idx]
Xva_sel = X_val[:, idx]
Xte_sel = X_test[:, idx]
# in questa parte vengono tagliate le colonne del dataset per comprendere solo le k features.

# Manual grid search again but now restricted to the selected features
best_score_sel = -np.inf
best_params_sel = None

for C in C_grid:
    for gamma in gamma_grid:
        pipe = svm_pipeline(C, gamma)
        pipe.fit(Xtr_sel, y_train)      # train on TRAIN
        val_acc = pipe.score(Xva_sel, y_val)  # validate on VAL
        if val_acc > best_score_sel:
            best_score_sel = val_acc
            best_params_sel = {"C": C, "gamma": gamma}
# visto che questo è il modello finale, direi di scegliere una grid piu ampia, scegliendo come base i valori 
# di c e gamma ottenuti nelle varie run di cross validation

#TRAINING FINALE DEL SVM

# Train final model on TRAIN+VAL with best params (optional) or just TRAIN; here we keep TRAIN only as per your outline
final_pipe = svm_pipeline(best_params_sel["C"], best_params_sel["gamma"])
final_pipe.fit(Xtr_sel, y_train)
test_acc = final_pipe.score(Xte_sel, y_test)

print("Selected features (best k):", best_subset)
print("Best validation accuracy on selected features:", f"{best_score_sel:.3f}", "with", best_params_sel)
print("Test accuracy (selected features, tuned on val):", f"{test_acc:.3f}")

# For comparison: test accuracy with all features using baseline best params
baseline_pipe = svm_pipeline(best_params_base["C"], best_params_base["gamma"])
baseline_pipe.fit(X_train, y_train)
test_acc_all = baseline_pipe.score(X_test, y_test)
print("Test accuracy (all features, baseline tuned on val):", f"{test_acc_all:.3f}")